# 分析负载中的周期性


## AzureFunction 2021

In [11]:
# 我们已经处理好了每一个函数的负载数据，现在我们来分析一下这些数据的周期性
from smart_pred.dataset.azure_trace_2021 import AzureFunction2021

In [12]:
dataset = AzureFunction2021()
dataset.init()

In [13]:
app_list = dataset.get_app_list()

In [14]:
from py_plotter.plot import Plotter

my_plotter = Plotter(
    figsize=(20,6),
    fontsize=30,
)

In [15]:
def plot_trace_with_app(app_name, output_dir="./output"):
    # 先创建一个output_dir
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 在output_dir下创建一个app_name的文件夹
    if not os.path.exists(f'{output_dir}/{app_name}'):
        os.makedirs(f'{output_dir}/{app_name}')
    
    for function in dataset.get_function_list_by_app(app_name):
        # 获取trace
        trace = dataset.get_app_function_avg_concurrency_in_sec_list(app_name=app_name, function_name=function)
        
        # 在app_name下创建一个function_name的文件夹
        if not os.path.exists(f'{output_dir}/{app_name}/{function}'):
            os.makedirs(f'{output_dir}/{app_name}/{function}')

        # 分别画14天中的每一天的
        for day in range(14):
            # 画图
            x = [i for i in range(24*60*60)]
            y = trace[day*24*60*60:(day+1)*24*60*60]
            line_data_list = []
            x_list = []
            x_list.append(x)
            line_data_list.append(y)
            
            save_name = f"{function}-{day}.pdf"
            save_root = f'{output_dir}/{app_name}/{function}'
            
            my_plotter.plot_lines(
                x_list=x_list,
                x_tick_ndigits=0,
                y_tick_ndigits=1,
                line_data_list=line_data_list,
                legend_title=None,
                legend_label_list=["ts"],
                x_label='Timestamp (s) in a day',
                y_label='Concurrency',
                title=None,
                save_root=save_root,
                filename=save_name,
            )
            
            print(f"save to {save_root}")
        
        

In [6]:
# 画前5个app的
for app in app_list[:5]:
    plot_trace_with_app(app_name=app, output_dir="./output/azure_function_2021")

pickle文件 /Users/wzj/GitHubProjects/smart-pred/datasets/AzureFunctionsInvocationTraceForTwoWeeksJan2021/avg_concurrency_in_sec_by_app/a9fd6223e00378edd60a03c721f9345f34d198757a4057b349d0e85e3aa111b2.pickle 存在，直接读取
图片保存到:./output/azure_function_2021/a9fd6223e00378edd60a03c721f9345f34d198757a4057b349d0e85e3aa111b2/6aee0815ebb60057503e8b25bb2685ed896d9faf9925e042de8c09bd1e1f8ecc/6aee0815ebb60057503e8b25bb2685ed896d9faf9925e042de8c09bd1e1f8ecc-0.pdf
save to ./output/azure_function_2021/a9fd6223e00378edd60a03c721f9345f34d198757a4057b349d0e85e3aa111b2/6aee0815ebb60057503e8b25bb2685ed896d9faf9925e042de8c09bd1e1f8ecc
图片保存到:./output/azure_function_2021/a9fd6223e00378edd60a03c721f9345f34d198757a4057b349d0e85e3aa111b2/6aee0815ebb60057503e8b25bb2685ed896d9faf9925e042de8c09bd1e1f8ecc/6aee0815ebb60057503e8b25bb2685ed896d9faf9925e042de8c09bd1e1f8ecc-1.pdf
save to ./output/azure_function_2021/a9fd6223e00378edd60a03c721f9345f34d198757a4057b349d0e85e3aa111b2/6aee0815ebb60057503e8b25bb2685ed896d9faf9925e0

In [16]:
# 筛选最大concurency=1的function
def get_max_concurency_function(max_concurency=1):
    # 先获取全部的function_list
    function_list = dataset.get_function_list()
    
    # 然后筛选
    
    result_dict_list = []
    for function in function_list:
        dataset.get_app_by_function(function_name=function)
        app_name = dataset.get_app_by_function(function_name=function)
        trace = dataset.get_app_function_avg_concurrency_in_sec_list(app_name=app_name, function_name=function)
        if max(trace) == max_concurency:
            print(f"{function} has max concurency {max_concurency}")
            # 找到该function的app_name
            
            
            result_dict = {
                "app_name": app_name,
                "function_name": function,
                "max_concurency": max_concurency,
            }
            result_dict_list.append(result_dict)
            
    return result_dict_list        

In [17]:

def find_function_name_list_with_max_concurency_in_range(cmax_min=4, cmax_max=4):
    # 先获取全部的function_list
    function_list = dataset.get_function_list()
    # 然后筛选
    result_dict_list = []
    for function in function_list:
        dataset.get_app_by_function(function_name=function)
        app_name = dataset.get_app_by_function(function_name=function)
        trace = dataset.get_app_function_avg_concurrency_in_sec_list(app_name=app_name, function_name=function)
        max_concurency = max(trace)
        if max(trace) >= cmax_min and max(trace) <= cmax_max:
            print(f"{function} has max concurency {max_concurency}")
            # 找到该function的app_name
            result_dict = {
                "app_name": app_name,
                "function_name": function,
                "max_concurency": max_concurency,
            }
            result_dict_list.append(result_dict)
    
    # 最大concurency=1的function的app_name一共
    len(result_dict_list)
    
    # 从result_dict_list中获取function_name_list
    function_name_list = [result_dict["function_name"] for result_dict in result_dict_list]
    
    # 保存为txt
    filename = f"./output/azure_function_2021/max_concurency_{cmax_min}_{cmax_max}_function_name_list.txt"
    with open(filename, "w") as f:
        for function_name in function_name_list:
            f.write(f"{function_name}\n")

In [18]:
find_function_name_list_with_max_concurency_in_range(cmax_min=5, cmax_max=1000)

pickle文件 /Users/wzj/GitHubProjects/smart-pred/datasets/AzureFunctionsInvocationTraceForTwoWeeksJan2021/avg_concurrency_in_sec_by_app/96149d3ed4f00afb92f12856101e693e93c4683f030ae6ca6ebfbb7ba5a2a345.pickle 存在，直接读取
pickle文件 /Users/wzj/GitHubProjects/smart-pred/datasets/AzureFunctionsInvocationTraceForTwoWeeksJan2021/avg_concurrency_in_sec_by_app/dfcd50e431bff00cbb9b2fb9e3e6f312fab091b64488a46c6dffe525830aacea.pickle 存在，直接读取
pickle文件 /Users/wzj/GitHubProjects/smart-pred/datasets/AzureFunctionsInvocationTraceForTwoWeeksJan2021/avg_concurrency_in_sec_by_app/a83894b2184e57e722b73e2ffef16f2efa69307f409232ca36a02fe03bca2553.pickle 存在，直接读取
pickle文件 /Users/wzj/GitHubProjects/smart-pred/datasets/AzureFunctionsInvocationTraceForTwoWeeksJan2021/avg_concurrency_in_sec_by_app/dfcd50e431bff00cbb9b2fb9e3e6f312fab091b64488a46c6dffe525830aacea.pickle 存在，直接读取
pickle文件 /Users/wzj/GitHubProjects/smart-pred/datasets/AzureFunctionsInvocationTraceForTwoWeeksJan2021/avg_concurrency_in_sec_by_app/4f931629ab2497d

In [21]:
# 一共的function数目
len(dataset.get_function_list())

424

In [22]:
# 一共的app数目
len(dataset.get_app_list())

119

In [20]:
def plot_trace_with_function(function_name, output_dir="./output/function"):
    # 先创建一个output_dir
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 在output_dir下创建一个function_name的文件夹
    if not os.path.exists(f'{output_dir}/{function_name}'):
        os.makedirs(f'{output_dir}/{function_name}')
        # get app_name
        app_name = dataset.get_app_by_function(function_name=function_name)
    
        # 获取trace
        trace = dataset.get_app_function_avg_concurrency_in_sec_list(app_name=app_name, function_name=function_name)

        # 分别画14天中的每一天的
        for day in range(14):
            # 画图
            x = [i for i in range(24*60*60)]
            y = trace[day*24*60*60:(day+1)*24*60*60]
            line_data_list = []
            x_list = []
            x_list.append(x)
            line_data_list.append(y)
            
            save_name = f"{function_name}-{day}.pdf"
            save_root = f'{output_dir}/{function_name}'
            
            my_plotter.plot_lines(
                x_list=x_list,
                x_tick_ndigits=0,
                y_tick_ndigits=1,
                line_data_list=line_data_list,
                legend_title=None,
                legend_label_list=["ts"],
                x_label='Timestamp (s) in a day',
                y_label='Concurrency',
                title=None,
                save_root=save_root,
                filename=save_name,
            )
            
            print(f"save to {save_root}")

In [22]:
# 画 5-1000的 function plot_trace_with_app(app_name=app, output_dir="./output/azure_function_2021")
txt_path = "~/GitHubProjects/smart-pred/measurement/trace_analysis/output/azure_function_2021/max_concurency_5_1000_function_name_list.txt"
# 转换为绝对路径
import os
txt_path = os.path.expanduser(txt_path)
# 读取txt
with open(txt_path, "r") as f:
    function_name_list = f.readlines()
    function_name_list = [function_name.strip() for function_name in function_name_list]
    
for function_name in function_name_list:
    plot_trace_with_function(function_name=function_name, output_dir="./output/azure_function_2021/max_concurency_5_1000_function")

pickle文件 /Users/wzj/GitHubProjects/smart-pred/datasets/AzureFunctionsInvocationTraceForTwoWeeksJan2021/avg_concurrency_in_sec_by_app/fc4cdb6f79037e080699671080a27ffce1be75872c1cd39107a0ec92969a5f89.pickle 存在，直接读取
图片保存到:./output/azure_function_2021/max_concurency_5_1000_function/93002245db03177e592d0d08879dc855579cf7005733e2c66af36f1fe6221594/93002245db03177e592d0d08879dc855579cf7005733e2c66af36f1fe6221594-0.pdf
save to ./output/azure_function_2021/max_concurency_5_1000_function/93002245db03177e592d0d08879dc855579cf7005733e2c66af36f1fe6221594
图片保存到:./output/azure_function_2021/max_concurency_5_1000_function/93002245db03177e592d0d08879dc855579cf7005733e2c66af36f1fe6221594/93002245db03177e592d0d08879dc855579cf7005733e2c66af36f1fe6221594-1.pdf
save to ./output/azure_function_2021/max_concurency_5_1000_function/93002245db03177e592d0d08879dc855579cf7005733e2c66af36f1fe6221594
图片保存到:./output/azure_function_2021/max_concurency_5_1000_function/93002245db03177e592d0d08879dc855579cf7005733e2c66af

In [23]:
# 画 2的 function plot_trace_with_app(app_name=app, output_dir="./output/azure_function_2021")
txt_path = "~/GitHubProjects/smart-pred/measurement/trace_analysis/output/azure_function_2021/max_concurency_2_function_name_list.txt"
# 转换为绝对路径
import os
txt_path = os.path.expanduser(txt_path)
# 读取txt
with open(txt_path, "r") as f:
    function_name_list = f.readlines()
    function_name_list = [function_name.strip() for function_name in function_name_list]
    
for function_name in function_name_list:
    plot_trace_with_function(function_name=function_name, output_dir="./output/azure_function_2021/max_concurency_2_function")

pickle文件 /Users/wzj/GitHubProjects/smart-pred/datasets/AzureFunctionsInvocationTraceForTwoWeeksJan2021/avg_concurrency_in_sec_by_app/fcc7dff74ceb0debf03599051a7772fc8f7749b80d03bd4ac1f65abeab162855.pickle 存在，直接读取
图片保存到:./output/azure_function_2021/max_concurency_2_function/102992e9e972d134ec0139cff7bfa30e972fd7ab90823cbf9c67c8f28ee07723/102992e9e972d134ec0139cff7bfa30e972fd7ab90823cbf9c67c8f28ee07723-0.pdf
save to ./output/azure_function_2021/max_concurency_2_function/102992e9e972d134ec0139cff7bfa30e972fd7ab90823cbf9c67c8f28ee07723
图片保存到:./output/azure_function_2021/max_concurency_2_function/102992e9e972d134ec0139cff7bfa30e972fd7ab90823cbf9c67c8f28ee07723/102992e9e972d134ec0139cff7bfa30e972fd7ab90823cbf9c67c8f28ee07723-1.pdf
save to ./output/azure_function_2021/max_concurency_2_function/102992e9e972d134ec0139cff7bfa30e972fd7ab90823cbf9c67c8f28ee07723
图片保存到:./output/azure_function_2021/max_concurency_2_function/102992e9e972d134ec0139cff7bfa30e972fd7ab90823cbf9c67c8f28ee07723/102992e9e97